In [1]:
import torch
from torch import nn
from torch.nn import Module, Sequential, Linear, ReLU, Dropout,Sigmoid
import sys
import numpy as np
import pandas as pd
import scanpy as sc
import os
import re
import random
import pickle
import scipy.stats as stats
from anndata import AnnData
import pingouin as pg
import matplotlib.pyplot as plt
import math
from collections import defaultdict
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Processing data

In [10]:
annotation_data=pd.read_table('./dataset/unc_v2_exon_hg19_probe_TCGA.txt',index_col=0)
gene_dic=defaultdict(lambda:'unknown')
gene_to_id_dic={}
lab=0
for i in tqdm(range(len(annotation_data.index))):
    id=annotation_data.index[i]
    gene=annotation_data['gene'][i]
    if isinstance(gene,str):
        gene_dic[id]=gene
        if lab==0:
            for each in gene.split(","):
                if each not in gene_to_id_dic:
                    gene_to_id_dic[each]=[]
                gene_to_id_dic[each].append(id)

100%|██████████████████████████████████████████████████████████████████████| 208970/208970 [00:01<00:00, 178931.26it/s]


In [19]:
data=pd.read_table('./dataset/HiSeqV2_exon',index_col=0)
#X染色体编号设为23 Y染色体编号设为24
Obs_data={'gene':data.index.map(lambda x:gene_dic[x]),'chr':data.index.map(lambda x:ord(re.split("[:-]",x)[0][3:4])-65 if not (re.split("[:-]",x)[0][3:]).isdigit() else int(re.split("[:-]",x)[0][3:])),'chr_sta':data.index.map(lambda x:int(re.split("[:-]",x)[1]))}
Obs=pd.DataFrame(Obs_data,index=data.index,columns=['gene','chr','chr_sta'])
Obs=Obs.sort_values(by=['chr','chr_sta'],ascending=[True,True])
adata=sc.AnnData(data.reindex(Obs.index),obs=Obs)
adata.write('./dataset/anndata1.h5ad')

... storing 'gene' as categorical


# Load data

In [3]:
adata=sc.read_h5ad('./dataset/anndata.h5ad')
#https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.BRCA.sampleMap%2FHiSeqV2_exon.gz 

In [3]:
adata.obs

,gene,chr,chr_sta
sample,,,
chr1:11874-12227:+,DDX11L1,1,11874
chr1:12595-12721:+,DDX11L1,1,12595
chr1:12613-12721:+,unknown,1,12613
chr1:12646-12697:+,unknown,1,12646
chr1:13221-14409:+,DDX11L1,1,13221
...,...,...,...
chrY:27876939-27877073:+,TTTY3,24,27876939
chrY:27879431-27879535:+,TTTY3,24,27879431
chrY:28121660-28121764:+,PRYP4,24,28121660


In [4]:
F=open('TCGA_ChiTaRS_combined_fusion_information_on_hg19.txt','r')

In [5]:
pairs=[]
i=0
for e in F:
    l=e.split()
    if l[2]!=l[6]:
        if (l[2],l[6]) not in pairs:
            pairs.append((l[2],l[6]))
print(len(pairs))

42917


In [6]:
F2=pd.read_excel('ChimerSeq4.xlsx')

In [7]:
for i in range(len(F2)):
    if F2['H_gene'][i]!=F2['T_gene'][i]:
        if (F2['H_gene'][i],F2['T_gene'][i]) not in pairs:
            pairs.append((F2['H_gene'][i],F2['T_gene'][i]))
print(len(pairs))

96337


In [8]:
alr=[]
for each in adata.obs['gene']:
    for each2 in each.split(','):
        if each2 not in alr:
            alr.append(each2)
print(len(alr))

22765


In [9]:
alr2=[]
used_pairs=[]
for each in pairs:
    if (each[0] in alr) and (each[1] in alr):
        if each[0] not in alr2:
            alr2.append(each[0])
        if each[1] not in alr2:
            alr2.append(each[1])
        used_pairs.append((each[0],each[1]))
print(len(used_pairs),len(alr2))

58434 16366


In [10]:
ALR_X=[]
ALR_2=[]
for each in tqdm(alr2):
    temp=[]
    for i in range(len(adata.obs['gene'])):
        if each in adata.obs['gene'][i].split(','):
            if adata.X[i].mean()>0.1:
                temp.append(adata.X[i])
    if len(temp)!=0:
        ALR_2.append(each)
        ALR_X.append(temp)

100%|██████████████████████████████████████████████████████████████████████████| 16366/16366 [9:49:24<00:00,  2.16s/it]


In [11]:
len(ALR_X),len(ALR_2)

(15107, 15107)

In [12]:
np.save('ALR_X2.npy',ALR_X)
np.save('ALR_22.npy',ALR_2)

In [10]:
ALR_X=np.load('ALR_X2.npy',allow_pickle=True)
ALR_2=np.load('ALR_22.npy',allow_pickle=True)

In [ ]:
used_pairs2=[]
i=0
while 1:
    A=random.choice(ALR_2)
    B=random.choice(ALR_2)
    if (A,B) not in used_pairs2:
        if (A,B) not in used_pairs:
            if A!=B:
                used_pairs2.append((A,B))
                i+=1
    if i==len(used_pairs):
        break

In [ ]:
used_pairs_=[]
used_pairs2_=[]
for each in used_pairs:
    if each[0] in ALR_2 and each[1] in ALR_2:
        X0=ALR_X[ALR_2.index(each[0])]
        X1=ALR_X[ALR_2.index(each[1])]
        print(len(X0),len(X1))
        if len(X0)<=15 and len(X0)>2 and len(X1)<=15 and len(X1)>2:
            X=np.vstack((X0,X1))
            corr=np.array(pd.DataFrame(X.T).corr())
            lab=0
            for each2 in corr:
                if np.isnan(each2).any():
                    lab=1
            if lab==0:
                used_pairs_.append(each)
for each in used_pairs2:
    if each[0] in ALR_2 and each[1] in ALR_2:
        X0=ALR_X[ALR_2.index(each[0])]
        X1=ALR_X[ALR_2.index(each[1])]
        if len(X0)<=15 and len(X0)>2 and len(X1)<=15 and len(X1)>2:
            X=np.vstack((X0,X1))
            corr=np.array(pd.DataFrame(X.T).corr())
            lab=0
            for each2 in corr:
                if np.isnan(each2).any():
                    lab=1
            if lab==0:
                used_pairs2_.append(each)

In [ ]:
print(len(used_pairs_),len(used_pairs2_))

In [ ]:
counti=0
with open('FINAL.txt','w') as f:
    f.write('2\n')
    for i in range(500,530):
        for each in used_pairs_[i]:
            f.write(str(counti)+'_'+each+'\n')
            counti+=1
            for each2 in ALR_X[alr2.index(each)]:
                for each3 in each2:
                    f.write(str(each3)+'\t')
                f.write('\n')
    for i in range(500,530):
        for each in used_pairs2_[i]:
            f.write(str(counti)+'_'+each+'\n')
            counti+=1
            for each2 in ALR_X[alr2.index(each)]:
                for each3 in each2:
                    f.write(str(each3)+'\t')
                f.write('\n')
print(counti)

In [16]:
DATA=[]
MAX_LEN=30
device=torch.device('cuda:0')
for each in used_pairs_:
    X0=ALR_X[ALR_2.index(each[0])]
    if len(X0)>int(MAX_LEN/2):
        X0=X0[:int(MAX_LEN/2)]
    X1=ALR_X[ALR_2.index(each[1])]
    if len(X1)>int(MAX_LEN/2):
        X1=X1[:int(MAX_LEN/2)]
    Y=1
    X=np.vstack((X0,X1))
    L=len(X)+1
    pm=np.zeros((len(X),len(X)))
    for i in range(len(X)):
        pm[i][i]=1
    for i in range(len(X)):
        for j in range(i+1,len(X)):
            p=stats.pearsonr(X[i],X[j])[1]
            pm[i][j]=p
            pm[j][i]=p
    pm=np.vstack((pm[:len(X0)],np.zeros(len(pm[0])),pm[len(X0):]))
    pm=np.hstack((pm[:,:len(X0)],np.zeros([len(pm),1]),pm[:,len(X0):]))
    corr=np.array(pd.DataFrame(X.T).corr())
    corr=np.vstack((corr[:len(X0)],np.zeros(len(corr[0])),corr[len(X0):]))
    corr=np.hstack((corr[:,:len(X0)],np.zeros([len(corr),1]),corr[:,len(X0):]))
    pcorr=np.array(pd.DataFrame(X.T).pcorr())
    pcorr=np.vstack((pcorr[:len(X0)],np.zeros(len(pcorr[0])),pcorr[len(X0):]))
    pcorr=np.hstack((pcorr[:,:len(X0)],np.zeros([len(pcorr),1]),pcorr[:,len(X0):]))
    if len(pm[0])<MAX_LEN+1:
        pm=np.vstack((pm,np.zeros([MAX_LEN+1-len(pm),len(pm[0])])))
        corr=np.vstack((corr,np.zeros([MAX_LEN+1-len(corr),len(corr[0])])))
        pcorr=np.vstack((pcorr,np.zeros([MAX_LEN+1-len(pcorr),len(pcorr[0])])))
        pm=np.hstack((pm,np.zeros([MAX_LEN+1,MAX_LEN+1-len(pm[0])])))
        corr=np.hstack((corr,np.zeros([MAX_LEN+1,MAX_LEN+1-len(corr[0])])))
        pcorr=np.hstack((pcorr,np.zeros([MAX_LEN+1,MAX_LEN+1-len(pcorr[0])])))
    pm=torch.tensor(pm)
    corr=torch.tensor(corr)
    pcorr=torch.tensor(pcorr)
    L=torch.tensor(L)
    Y=torch.tensor(Y)
    pm=pm.to(torch.float32)
    corr=corr.to(torch.float32)
    pcorr=pcorr.to(torch.float32)
    Y=Y.to(torch.float32)
    pm=pm.to(device)
    corr=corr.to(device)
    pcorr=pcorr.to(device)
    Y=Y.to(device)
    DATA.append((corr,pcorr,pm,L,Y))
for each in used_pairs2_:
    X0=ALR_X[ALR_2.index(each[0])]
    if len(X0)>int(MAX_LEN/2):
        X0=X0[:int(MAX_LEN/2)]
    X1=ALR_X[ALR_2.index(each[1])]
    if len(X1)>int(MAX_LEN/2):
        X1=X1[:int(MAX_LEN/2)]
    Y=0
    X=np.vstack((X0,X1))
    L=len(X)+1
    pm=np.zeros((len(X),len(X)))
    for i in range(len(X)):
        pm[i][i]=1
    for i in range(len(X)):
        for j in range(i+1,len(X)):
            p=stats.pearsonr(X[i],X[j])[1]
            pm[i][j]=p
            pm[j][i]=p
    pm=np.vstack((pm[:len(X0)],np.zeros(len(pm[0])),pm[len(X0):]))
    pm=np.hstack((pm[:,:len(X0)],np.zeros([len(pm),1]),pm[:,len(X0):]))
    corr=np.array(pd.DataFrame(X.T).corr())
    corr=np.vstack((corr[:len(X0)],np.zeros(len(corr[0])),corr[len(X0):]))
    corr=np.hstack((corr[:,:len(X0)],np.zeros([len(corr),1]),corr[:,len(X0):]))
    pcorr=np.array(pd.DataFrame(X.T).pcorr())
    pcorr=np.vstack((pcorr[:len(X0)],np.zeros(len(pcorr[0])),pcorr[len(X0):]))
    pcorr=np.hstack((pcorr[:,:len(X0)],np.zeros([len(pcorr),1]),pcorr[:,len(X0):]))
    if len(pm[0])<MAX_LEN+1:
        pm=np.vstack((pm,np.zeros([MAX_LEN+1-len(pm),len(pm[0])])))
        corr=np.vstack((corr,np.zeros([MAX_LEN+1-len(corr),len(corr[0])])))
        pcorr=np.vstack((pcorr,np.zeros([MAX_LEN+1-len(pcorr),len(pcorr[0])])))
        pm=np.hstack((pm,np.zeros([MAX_LEN+1,MAX_LEN+1-len(pm[0])])))
        corr=np.hstack((corr,np.zeros([MAX_LEN+1,MAX_LEN+1-len(corr[0])])))
        pcorr=np.hstack((pcorr,np.zeros([MAX_LEN+1,MAX_LEN+1-len(pcorr[0])])))
    pm=torch.tensor(pm)
    corr=torch.tensor(corr)
    pcorr=torch.tensor(pcorr)
    L=torch.tensor(L)
    Y=torch.tensor(Y)
    pm=pm.to(torch.float32)
    corr=corr.to(torch.float32)
    pcorr=pcorr.to(torch.float32)
    Y=Y.to(torch.float32)
    pm=pm.to(device)
    corr=corr.to(device)
    pcorr=pcorr.to(device)
    Y=Y.to(device)
    DATA.append((corr,pcorr,pm,L,Y))

In [2]:
train_loader=torch.load('train_loader_FINAL.pth')
test_loader=torch.load('test_loader_FINAL.pth')

In [17]:
random.shuffle(DATA)
train_loader=torch.utils.data.DataLoader(dataset=DATA[:int(0.8*len(DATA))],batch_size=3,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=DATA[int(0.8*len(DATA)):],batch_size=3,shuffle=True)

In [18]:
torch.save(train_loader,'train_loader_FINAL.pth')
torch.save(test_loader,'test_loader_FINAL.pth')

# train

In [3]:
class MODEL(nn.Module):
    def __init__(self):
        super(MODEL,self).__init__()
        Hidden=16
        MAX_LEN=30
        self.corr_lstm=nn.LSTM(input_size=MAX_LEN+1,hidden_size=Hidden,num_layers=3,batch_first=True,bidirectional=True)
        self.pcorr_lstm=nn.LSTM(input_size=MAX_LEN+1,hidden_size=Hidden,num_layers=3,batch_first=True,bidirectional=True)
        self.pm_lstm=nn.LSTM(input_size=MAX_LEN+1,hidden_size=Hidden,num_layers=1,batch_first=True,bidirectional=True)
        #self.c_sequential=Sequential(Dropout(p=0.5),Linear(int(3*Hidden),Hidden),Linear(Hidden,1),Sigmoid())
        self.c_sequential=Sequential(Dropout(p=0.5),Linear(3*Hidden,1),Sigmoid())
    def forward(self,corr,pcorr,pm,L):
        packed_corr=torch.nn.utils.rnn.pack_padded_sequence(corr,L,batch_first=True,enforce_sorted=False)
        packed_pcorr=torch.nn.utils.rnn.pack_padded_sequence(pcorr,L,batch_first=True,enforce_sorted=False)
        packed_pm=torch.nn.utils.rnn.pack_padded_sequence(pm,L,batch_first=True,enforce_sorted=False)
        seq0,(ht0,ct0)=self.corr_lstm(packed_corr)
        seq1,(ht1,ct1)=self.pcorr_lstm(packed_pcorr)
        seq2,(ht2,ct2)=self.pm_lstm(packed_pm)
        x=torch.cat([ht0[-1],ht1[-1],ht2[-1]],dim=1)
        x=self.c_sequential(x).T[0]
        return x

In [4]:
def train(model,train_loader,loss_func,optimizer,device):
    model.train()
    loss=0
    for i,(corr,pcorr,pm,L,Y) in enumerate(train_loader):
        outputs=model(corr,pcorr,pm,L)
        temp_loss=loss_func(outputs,Y)
        loss+=temp_loss
    print('train loss:',loss.item()/len(train_loader))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()/len(train_loader)

In [5]:
def evaluate(model,val_loader,loss_func,device):
    model.eval()
    with torch.no_grad():
        correct=0
        total=0
        loss=0
        for i,(corr,pcorr,pm,L,Y) in enumerate(val_loader):
            outputs=model(corr,pcorr,pm,L)
            loss+=loss_func(outputs,Y)
            R=1-abs(Y-outputs)
            R[R>=0.5]=1
            R[R<0.5]=0
            correct+=int(R.sum())
            total+=Y.size(0)
        print('val loss:',loss.item()/len(val_loader))
        accuracy=correct/total
        print('Accuracy on Test Set: {:.4f} %'.format(100*accuracy))
        return accuracy

In [6]:
def evaluate2(model,val_loader,loss_func,device):
    model.eval()
    with torch.no_grad():
        correct=0
        total=0
        r1p1=0
        r1p0=0
        r0p1=0
        r0p0=0
        loss=0
        t_count=[0,0,0,0]#0.9,0.8,0.7,0.6
        f_count=[0,0,0,0]
        for i,(corr,pcorr,pm,L,Y) in enumerate(val_loader):
            outputs=model(corr,pcorr,pm,L)
            print('   real:',Y)
            loss+=loss_func(outputs,Y)
            R=1-abs(Y-outputs)
            R[R>=0.5]=1
            R[R<0.5]=0
            correct+=int(R.sum())
            total+=Y.size(0)
            
            for i in range(len(outputs)):
                if outputs[i]>=0.9:
                    if Y[i]==1:
                        t_count[3]+=1
                    else:
                        f_count[3]+=1
                elif outputs[i]>=0.8:
                    if Y[i]==1:
                        t_count[2]+=1
                    else:
                        f_count[2]+=1
                elif outputs[i]>=0.7:
                    if Y[i]==1:
                        t_count[1]+=1
                    else:
                        f_count[1]+=1
                elif outputs[i]>=0.6:
                    if Y[i]==1:
                        t_count[0]+=1
                    else:
                        f_count[0]+=1
            outputs[outputs>=0.5]=1
            outputs[outputs<0.5]=0
            for i in range(len(outputs)):
                if Y[i]==1:
                    if outputs[i]==1:
                        r1p1+=1
                    else:
                        r1p0+=1
                else:
                    if outputs[i]==1:
                        r0p1+=1
                    else:
                        r0p0+=1
            print('predict:',outputs)
        print('val loss:',loss.item()/len(val_loader))
        accuracy=correct/total
        print('Accuracy on Test Set: {:.4f} %'.format(100 * accuracy))
        precision=r1p1/(r1p1+r0p1)
        print('total:',total,' real=1,predict=1:',r1p1,' real=1,predict=0:',r1p0,' real=0,predict=1:',r0p1,' real=0,predict=0:',r0p0)
        print('Precision on Test Set: {:.4f} %'.format(100 * precision))
        print('Precision when above 0.6: {:.4f} %'.format(100 * t_count[0]/(t_count[0]+f_count[0])),' total num is ',str((t_count[0]+f_count[0])))
        print('Precision when above 0.7: {:.4f} %'.format(100 * t_count[1]/(t_count[1]+f_count[1])),' total num is ',str((t_count[1]+f_count[1])))
        print('Precision when above 0.8: {:.4f} %'.format(100 * t_count[2]/(t_count[2]+f_count[2])),' total num is ',str((t_count[2]+f_count[2])))
        print('Precision when above 0.9: {:.4f} %'.format(100 * t_count[3]/(t_count[3]+f_count[3])),' total num is ',str((t_count[3]+f_count[3])))
        return accuracy

In [7]:
def show_curve(ys,title):
    x=np.array(range(len(ys)))
    y=np.array(ys)
    plt.plot(x,y,c='b')
    plt.axis()
    plt.title('{} curve'.format(title))
    plt.xlabel('epoch')
    plt.ylabel('{}'.format(title))
    plt.show()

In [8]:
def fit(model,num_epochs,optimizer,device):
    loss_func=nn.MSELoss()
    loss_func=loss_func.to(device)
    losses=[]
    accs=[]
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch+1,num_epochs))
        loss=train(model,train_loader,loss_func,optimizer,device)
        losses.append(loss)
        if (epoch+1)%10==0:
            accuracy=evaluate(model,test_loader,loss_func,device)
            accs.append(accuracy)
            torch.save(model.state_dict(),'./FINAL_plus_'+str(epoch+1)+'.pkl')
    show_curve(losses,"train loss")
    show_curve(accs,"test accuracy")

In [ ]:
#hidden=16 layer=3 pvalue_layer=1 bidirectional=true
num_epochs=10000
lr=0.001
device=torch.device('cuda:0')
model=MODEL()
model=model.to(device)
load=0
if load==1:
    model.load_state_dict(torch.load('./FINAL_500.pkl'))
optimizer=torch.optim.Adam(model.parameters(),lr=lr) 
fit(model,num_epochs,optimizer,device)

Epoch 1/10000:
train loss: 0.24825885501702374
Epoch 2/10000:
train loss: 0.24817510910495977
Epoch 3/10000:
train loss: 0.24770969416389574
Epoch 4/10000:
train loss: 0.247642728544869
Epoch 5/10000:
train loss: 0.24718882453483396
Epoch 6/10000:
train loss: 0.24745211763798183
Epoch 7/10000:
train loss: 0.24691013167260673
Epoch 8/10000:
train loss: 0.24697818616348757
Epoch 9/10000:
train loss: 0.24680445277033888
Epoch 10/10000:
train loss: 0.24673136224713285
val loss: 0.24685640751309135
Accuracy on Test Set: 55.0812 %
Epoch 11/10000:
train loss: 0.24650172695076059
Epoch 12/10000:
train loss: 0.2462360227742215
Epoch 13/10000:
train loss: 0.24626310846159208
Epoch 14/10000:
train loss: 0.24593298584839918
Epoch 15/10000:
train loss: 0.24582380700088602
Epoch 16/10000:
train loss: 0.2454183772324984
Epoch 17/10000:
train loss: 0.24525362703214598
Epoch 18/10000:
train loss: 0.24472099369823191
Epoch 19/10000:
train loss: 0.24471276882690546
Epoch 20/10000:
train loss: 0.244283286

train loss: 0.22905628876383802
Epoch 153/10000:
train loss: 0.2288478864686206
Epoch 154/10000:
train loss: 0.2289989479942358
Epoch 155/10000:
train loss: 0.22886552230382884
Epoch 156/10000:
train loss: 0.22844753162083864
Epoch 157/10000:
train loss: 0.22844022062410402
Epoch 158/10000:
train loss: 0.22847957556397339
Epoch 159/10000:
train loss: 0.22819563278387822
Epoch 160/10000:
train loss: 0.22837344944647978
val loss: 0.2288342366821639
Accuracy on Test Set: 62.2385 %
Epoch 161/10000:
train loss: 0.22787785719974515
Epoch 162/10000:
train loss: 0.22802394102545595
Epoch 163/10000:
train loss: 0.22798678327343502
Epoch 164/10000:
train loss: 0.22799489147991997
Epoch 165/10000:
train loss: 0.22781019159166932
Epoch 166/10000:
train loss: 0.22797188906200222
Epoch 167/10000:
train loss: 0.22809917040408967
Epoch 168/10000:
train loss: 0.2277098987162512
Epoch 169/10000:
train loss: 0.22739146203134955
Epoch 170/10000:
train loss: 0.22761034471816263
val loss: 0.2286617088257108

Epoch 302/10000:
train loss: 0.22099237603038388
Epoch 303/10000:
train loss: 0.22100546193677326
Epoch 304/10000:
train loss: 0.22091461891351744
Epoch 305/10000:
train loss: 0.22071844698251833
Epoch 306/10000:
train loss: 0.22055457176847723
Epoch 307/10000:
train loss: 0.22138345602326576
Epoch 308/10000:
train loss: 0.2211181407295317
Epoch 309/10000:
train loss: 0.22084535888594098
Epoch 310/10000:
train loss: 0.22040617409007646
val loss: 0.2246488854929914
Accuracy on Test Set: 63.2473 %
Epoch 311/10000:
train loss: 0.2202362128601366
Epoch 312/10000:
train loss: 0.22046174544291575
Epoch 313/10000:
train loss: 0.22044138742807223
Epoch 314/10000:
train loss: 0.22034723890097363
Epoch 315/10000:
train loss: 0.2203853494158669
Epoch 316/10000:
train loss: 0.22030236182527277
Epoch 317/10000:
train loss: 0.2203577970691303
Epoch 318/10000:
train loss: 0.22007317374412633
Epoch 319/10000:
train loss: 0.22063841490124245
Epoch 320/10000:
train loss: 0.2208303480096667
val loss: 0.2

train loss: 0.2172950533173881
Epoch 452/10000:
train loss: 0.2176771501174737
Epoch 453/10000:
train loss: 0.21707797893561842
Epoch 454/10000:
train loss: 0.21675472010393437
Epoch 455/10000:
train loss: 0.2169805054605368
Epoch 456/10000:
train loss: 0.21647651334521345
Epoch 457/10000:
train loss: 0.2171265115096866
Epoch 458/10000:
train loss: 0.21683257444150206
Epoch 459/10000:
train loss: 0.2166015391670317
Epoch 460/10000:
train loss: 0.21625525847328966
val loss: 0.22580069802907973
Accuracy on Test Set: 63.6084 %
Epoch 461/10000:
train loss: 0.2164591497279687
Epoch 462/10000:
train loss: 0.21642735855865722
Epoch 463/10000:
train loss: 0.21621042028586931
Epoch 464/10000:
train loss: 0.21633674886508442
Epoch 465/10000:
train loss: 0.21631256035461133
Epoch 466/10000:
train loss: 0.21600763734718462
Epoch 467/10000:
train loss: 0.2163384405052863
Epoch 468/10000:
train loss: 0.21587360888842982
Epoch 469/10000:
train loss: 0.2162348421260254
Epoch 470/10000:
train loss: 0.2

In [10]:
device=torch.device('cuda:0')
model=MODEL()
model=model.to(device)
model.load_state_dict(torch.load('./FINAL_370.pkl'))
loss_func=nn.MSELoss()
loss_func=loss_func.to(device)
evaluate2(model,test_loader,loss_func,device)

   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([1., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 1.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([1., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 1.], device='cuda:0')
predict: tensor([0., 1., 1.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 1.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tens

   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([1., 0., 1.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([1., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 1.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 0., 1.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([1., 0., 1.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([1., 0., 1.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 0., 1.], device='cuda:0')
predict: tens

predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([1., 1., 0.], device='cuda:0')
   real: tensor([1., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([0., 1., 1.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([1., 1., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([1., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 1.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tens

predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([1., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([1., 1., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([0., 1., 1.], device='cuda:0')
   real: tens

predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([1., 0., 1.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 1.], device='cuda:0')
predict: tensor([1., 1., 0.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([1., 1., 0.], device='cuda:0')
   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tens

...

   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([1., 1., 1.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 0., 1.], device='cuda:0')
predict: tensor([1., 0., 1.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([1., 1., 1.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([1., 1., 0.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tens

   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 1., 1.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([0., 1., 1.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([1., 0., 1.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tens

   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 1.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([0., 1., 1.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([1., 1., 0.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([0., 1., 1.], device='cuda:0')
predict: tensor([0., 1., 1.], device='cuda:0')
   real: tensor([0., 0., 1.], device='cuda:0')
predict: tensor([0., 1., 1.], device='cuda:0')
   real: tensor([1., 0., 0.], device='cuda:0')
predict: tens

predict: tensor([0., 0., 1.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([1., 0., 1.], device='cuda:0')
   real: tensor([0., 1., 1.], device='cuda:0')
predict: tensor([1., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([0., 1., 0.], device='cuda:0')
predict: tensor([1., 1., 0.], device='cuda:0')
   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([1., 1., 0.], device='cuda:0')
predict: tensor([0., 1., 0.], device='cuda:0')
   real: tensor([0., 0., 0.], device='cuda:0')
predict: tensor([0., 0., 0.], device='cuda:0')
   real: tensor([1., 1., 1.], device='cuda:0')
predict: tensor([0., 0., 1.], device='cuda:0')
   real: tens

0.636296060316449